In [2]:
load('./data/hw5ds1.rdata')

In [233]:
testfold = 1
dstrain1 = hw5ds1[-folds[[testfold]],]
# dstrain1
filter_chisq <- function(dstrain, ypos="pos", min_count=5, chi_threshold=10**(-5)){
    pos_col = names(dstrain1)[1]
    new_cols = colSums(dstrain1[,2:dim(dstrain1)[2]] != 0)
    features = c()
    for(name in names(new_cols)){
       if(new_cols[name] > min_count){
            features <- c(features, name)
       }
    }
    new_df = dstrain1[c(pos_col,features)]
    N = nrow(new_df)
    chi_df<-data.frame(feat = character(), chisq_value = double())
    chi_sq_values = c()
    col_idx_array = c()
    for(feat in features){
        col_idx = grep(feat, colnames(new_df))
        col_idx_array = c(col_idx_array, grep(feat, colnames(dstrain1)))
        gs = c(0, 1)
        cs = c( "%notpositivetag%", ypos)
        present_or_not = 0
        on_topic_or_not = 0 
        chi_sq_sum = 0
        for(g in gs){
            for(c in cs){
                obsv_freq = 0
                if(g == 0){
                    present_or_not = nrow(new_df[new_df[, col_idx] == 0, ][feat])
                    if(c == ypos){
                       obsv_freq =  nrow(new_df[new_df[, 1] == ypos & new_df[, col_idx] == 0, ][feat])
                    }
                    else{
                       obsv_freq =  nrow(new_df[new_df[, 1] != ypos & new_df[, col_idx] == 0, ][feat])
                    }
                }
                else{
                    present_or_not = nrow(new_df[new_df[, col_idx] > 0, ][feat])
                    if(c == ypos){
                       obsv_freq =  nrow(new_df[new_df[, 1] == ypos & new_df[, col_idx] > 0, ][feat])
                    }
                    else{
                       obsv_freq =  nrow(new_df[new_df[, 1] != ypos & new_df[, col_idx] > 0, ][feat])

                    }
                }
                if(c == ypos){
                    on_topic_or_not =  nrow(new_df[new_df[, 1] == ypos, ][feat])
                }
                else{
                    on_topic_or_not =  nrow(new_df[new_df[, 1] !=ypos, ][feat])
                }
                E = present_or_not * on_topic_or_not / N
                chi_sq = (obsv_freq - E) **2 / E
                chi_sq_sum = chi_sq_sum + chi_sq
            }
        }
        chi_sq_values = c(chi_sq_values, chi_sq_sum )


    }
    chi_df <- data.frame(features, chi_sq_values, col_idx = col_idx_array)
    chi_df = chi_df[with(chi_df, order(-1 * chi_sq_values)), ]
    chi_df = chi_df[chi_df$chi_sq_values > chi_threshold,]
    result = list(colpos = chi_df$col_idx, colname = as.character((chi_df$features)),chistat = chi_df$chi_sq_values )
    if(nrow(chi_df) == 0){
        result = list(colpos = NULL, colname = NULL,chistat = NULL)
    }
    return(result)
}

testfold = 2
dstrain1 = hw5ds1[-folds[[testfold]],]
out1 = filter_chisq(dstrain)
head(out1$colpos, n=15)
head(out1$colname, n=15)
head(out1$chistat, n=15)

[1] 4111 4061 3150 4032 3806 2999 3783 3309 3510 2985 3503 3718 3812 3644 2953

[1] "w4110_product"      "w4060_new"          "w3149_launch"      
 [4] "w4031_development"  "w3805_growth"       "w2998_introduction"
 [7] "w3782_effort"       "w3308_enhance"      "w3509_expand"      
[10] "w2984_introduce"    "w3502_opportunity"  "w3717_develop"     
[13] "w3811_improve"      "w3643_competitive"  "w2952_family"

[1] 293.97045 202.75048  97.97374  80.13614  46.60871  41.07994  39.74142
 [8]  35.88621  35.25097  34.79360  30.45154  30.45154  29.70390  24.93498
[15]  24.65999

In [68]:
entropy <- function(p) { 
    if(p == 1 || p  < 10 ** (-6)) return(0)
    return(-1 * p * log2(p) - (1-p) * log2(1-p)) 
}

filter_ig <- function(dstrain, ypos="pos", min_count=5, ig_threshold=10**(-5)){
    pos_col = names(dstrain1)[1]
    new_cols = colSums(dstrain1[,2:dim(dstrain1)[2]] != 0)
    features = c()
    for(name in names(new_cols)){
       if(new_cols[name] > min_count){
            features <- c(features, name)
       }
    }
    new_df = dstrain1[c(pos_col,features)]
    N = nrow(new_df)
    N_POS = nrow(new_df[new_df[, 1] == ypos, ][1])
    H_Y = entropy(N_POS / N)
    ig_values_array = c()
    col_idx_array = c()
    for(feat in features){
        col_idx = grep(feat, colnames(new_df))
        col_idx_array = c(col_idx_array, grep(feat, colnames(dstrain1)))
        pos_present = nrow(new_df[new_df[, 1] == ypos & new_df[, col_idx] > 0, ][feat])
        pos_absent = nrow(new_df[new_df[, 1] == ypos & new_df[, col_idx] == 0, ][feat])
        neg_present = nrow(new_df[new_df[, 1] != ypos & new_df[, col_idx] > 0, ][feat])
        neg_absent = nrow(new_df[new_df[, 1]  != ypos & new_df[, col_idx] == 0, ][feat])
        p_hasfeat = (pos_present + neg_present) /N
        p_nofeat = (pos_absent + neg_absent)/ N
        p_has_feat_pos = pos_present / (pos_present + neg_present)
        p_no_feat_pos = pos_absent / (pos_absent + neg_absent)
        H_YX = p_hasfeat * entropy(p_has_feat_pos)  + p_nofeat* entropy(p_no_feat_pos)
        ig = H_Y - H_YX
        ig_values_array = c(ig_values_array, ig)
    }
    ig_df = data.frame(fname = features, ig_value = ig_values_array, col_idx = col_idx_array)
    ig_df = ig_df[with(ig_df, order(-1 * ig_value)), ]
    ig_df = ig_df[ig_df$ig_value > ig_threshold,]
    result = list(colpos = ig_df$col_idx, colname = as.character((ig_df$fname)),igvalue = ig_df$ig_value)
    if(nrow(ig_df) == 0){
        result = list(colpos = NULL, colname = NULL,igvalue = NULL)
    }
    return(result)
} 

testfold = 1
dstrain1 = hw5ds1[-folds[[testfold]],]
result = filter_ig(dstrain1)

In [69]:
print(head(result$colname, n=15))
print(head(result$igvalue, n=15))


 [1] "w4110_product"      "w4060_new"          "w3149_launch"      
 [4] "w4031_development"  "w2984_introduce"    "w3509_expand"      
 [7] "w3782_effort"       "w3805_growth"       "w3415_open"        
[10] "w2952_family"       "w3936_technology"   "w1496_nuvigil"     
[13] "w2998_introduction" "w3502_opportunity"  "w3635_achieve"     
 [1] 0.24983760 0.17714788 0.08660127 0.04862331 0.03489284 0.03442428
 [7] 0.03087182 0.03009023 0.02940856 0.02677397 0.02563883 0.02514924
[13] 0.02514924 0.01987746 0.01911226
